# Ollama + OpenAI + Python

## 1. Adja meg a modell nevét

Ha egy másik modellt töltött be, mint a "phi3:mini", módosítsa az alábbi cellában lévő értéket. Ez a változó a jegyzetfüzetben található kódokban lesz használva.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Az OpenAI kliens beállítása

Az OpenAI kliens általában az OpenAI.com vagy az Azure OpenAI segítségével használható, hogy nagy nyelvi modellekkel kommunikáljon.  
Azonban az Ollama-val is használható, mivel az Ollama egy OpenAI-kompatibilis végpontot biztosít a "http://localhost:11434/v1" címen.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Hozz létre egy csevegési választ

Most az OpenAI SDK segítségével létrehozhatunk egy választ egy beszélgetéshez. Ez a kérés egy haikut kell, hogy generáljon a macskákról:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Prompt tervezés

Az első üzenetet, amelyet a nyelvi modellnek küldünk, "rendszerüzenetnek" vagy "rendszerpromptnak" nevezzük, és ez határozza meg a modell számára az általános utasításokat.  
Lehetőséged van saját egyedi rendszerpromptot megadni, hogy irányítsd a nyelvi modellt, és másfajta kimenetet generáltass vele.  
Módosítsd az alábbi `SYSTEM_MESSAGE`-t, hogy a modell a kedvenc híres filmes/sorozatbeli karaktered stílusában válaszoljon, vagy meríts inspirációt más rendszerprompthoz az [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) oldalról.

Miután testre szabtad a rendszerüzenetet, add meg az első felhasználói kérdést a `USER_MESSAGE` mezőben.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Néhány példa

Egy másik módja annak, hogy irányítsunk egy nyelvi modellt, ha "néhány példát" adunk meg, azaz egy sor kérdés/válasz példát, amelyek bemutatják, hogyan kellene reagálnia.

Az alábbi példa azt próbálja elérni, hogy a nyelvi modell úgy viselkedjen, mint egy tanársegéd, néhány olyan kérdés és válasz példájának megadásával, amelyeket egy tanársegéd adhat, majd egy olyan kérdéssel ösztönzi a modellt, amelyet egy diák tehetne fel.

Próbáld ki először, majd módosítsd a `SYSTEM_MESSAGE`, `EXAMPLES` és `USER_MESSAGE` értékeket egy új helyzethez.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Információ-visszakeresésen alapuló generálás

A RAG (Retrieval Augmented Generation) egy olyan technika, amely lehetővé teszi, hogy egy nyelvi modell pontosan válaszoljon egy adott területre vonatkozó kérdésekre. Ehhez először releváns információkat keres egy tudásforrásból, majd az alapján generál választ.

Egy helyi CSV fájlt biztosítottunk, amely hibrid autókról tartalmaz adatokat. Az alábbi kód beolvassa a CSV fájlt, megkeresi a felhasználói kérdéshez illeszkedő találatokat, majd az így talált információk alapján generál választ. Fontos megjegyezni, hogy ez hosszabb időt vesz igénybe, mint az előző példák, mivel több adatot küld a modellnek. Ha azt tapasztalod, hogy a válasz még mindig nem az adatokon alapul, próbálkozhatsz rendszertervezéssel vagy más modellekkel. Általában a RAG hatékonyabb nagyobb modellekkel vagy finomhangolt SLM-ekkel.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
